# Clan Tournament System (Jupyter Style Overview)

In [ ]:
from django.db import models
from django.contrib.auth.models import User
from django.conf import settings
import os
import json
from PIL import Image

In [ ]:
class Clans(models.Model):
    clan_name = models.CharField(max_length=255)
    user = models.OneToOneField(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.clan_name

In [ ]:
class ClanTournament(models.Model):
    TOUR_CHOICES = [('league', 'League'), ('cup', 'Cup'), ('groups_knockout', 'Groups + Knockout')]
    PLAYER_MODE_CHOICES = [('fixed', 'Fixed'), ('dynamic', 'Dynamic')]

    name = models.CharField(max_length=255)
    description = models.TextField(blank=True, null=True)
    created_by = models.ForeignKey(User, on_delete=models.CASCADE)
    teams = models.ManyToManyField(Clans, related_name="clans")
    logo = models.ImageField(default="tours-defualt.jpg", upload_to='tour_logos')
    tour_type = models.CharField(max_length=100, choices=TOUR_CHOICES, blank=True, null=True)
    player_mode = models.CharField(max_length=10, choices=PLAYER_MODE_CHOICES, default='dynamic')

    def get_json_file_path(self):
        directory = os.path.join(settings.MEDIA_ROOT, 'tournament_data')
        os.makedirs(directory, exist_ok=True)
        return os.path.join(directory, f'tournament_clan_{self.pk}.json')

    def load_match_data_from_file(self):
        file_path = self.get_json_file_path()
        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
            with open(file_path, 'r') as json_file:
                try:
                    return json.load(json_file)
                except json.JSONDecodeError:
                    return {}
        return {}

    def save_match_data_to_file(self, match_data):
        with open(self.get_json_file_path(), 'w') as json_file:
            json.dump(match_data, json_file)

    def delete(self, *args, **kwargs):
        file_path = self.get_json_file_path()
        if os.path.exists(file_path):
            os.remove(file_path)
        super().delete(*args, **kwargs)

    def toggle_player_mode(self):
        self.player_mode = 'fixed' if self.player_mode == 'dynamic' else 'dynamic'
        self.save()

    def get_team_names(self):
        return [team.clan_name for team in self.teams.all()]

    def __str__(self):
        return self.name

In [ ]:
class ClanTournamentPlayer(models.Model):
    clan = models.ForeignKey(Clans, on_delete=models.CASCADE)
    tournament = models.ForeignKey(ClanTournament, on_delete=models.CASCADE)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    class Meta:
        unique_together = ('clan', 'tournament', 'user')

    def __str__(self):
        return f"{self.user.username} in {self.clan.clan_name} - {self.tournament.name}"

In [ ]:
from django.http import HttpResponseForbidden
from django.shortcuts import get_object_or_404, redirect

def assign_user_to_tournament(request, tournament_id, clan_id):
    if request.method == "POST":
        user_id = request.POST.get('user_id')
        user = get_object_or_404(User, pk=user_id)
        tournament = get_object_or_404(ClanTournament, pk=tournament_id)
        clan = get_object_or_404(Clans, pk=clan_id)

        if tournament.player_mode == 'fixed':
            return HttpResponseForbidden("Player assignments are fixed.")

        ClanTournamentPlayer.objects.get_or_create(user=user, clan=clan, tournament=tournament)
        return redirect('some-success-page')

In [ ]:
def get_clan_tournament_history(clan_id):
    return ClanTournamentPlayer.objects.filter(clan_id=clan_id).select_related("tournament")

In [ ]:
from django.db.models.signals import post_save

User.add_to_class('is_organizer', models.BooleanField(default=False))

def is_organizer(user):
    return getattr(user, 'is_organizer', False)